<a href="https://colab.research.google.com/github/gazon1/Recco-challenge/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from sklearn.metrics import # Rekko challenge 2019

```
                           /$$$$$$$  /$$$$$$$$ /$$   /$$ /$$   /$$  /$$$$$$ 
                          | $$__  $$| $$_____/| $$  /$$/| $$  /$$/ /$$__  $$
                          | $$  \ $$| $$      | $$ /$$/ | $$ /$$/ | $$  \ $$
                          | $$$$$$$/| $$$$$   | $$$$$/  | $$$$$/  | $$  | $$
                          | $$__  $$| $$__/   | $$  $$  | $$  $$  | $$  | $$
                          | $$  \ $$| $$      | $$\  $$ | $$\  $$ | $$  | $$
                          | $$  | $$| $$$$$$$$| $$ \  $$| $$ \  $$|  $$$$$$/
                          |__/  |__/|________/|__/  \__/|__/  \__/ \______/ 
                                                                            
```

Добро пожаловать на соревнование по машинному обучению от онлайн-кинотеатра [Okko](http://okko.tv) Rekko Challenge 2019.

В этом ноутбуке мы покажем вам пример простого но полного решения, от загрузки данных до формирования ответа. Для работы нам понадобятся библиотеки `pandas`, `numpy`, `scipy`, `implicit`, `pprint`, `tqdm`. Установить их в вашем рабочем окружении можно следующей командой.
```
pip install pandas numpy scipy implicit pprint tqdm
```

In [0]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from sklearn.metrics import average_precision_score as score

from pprint import pprint
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip3 install implicit

    100% |████████████████████████████████| 788kB 9.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/6c/e7/54f2a48bd3bd8a90f5b456a3e7b2ee687e111dbe25fc9fbf9d
Successfully built implicit


### Загрузка данных

Замените `DATA_PATH` на путь к данным, которые вы скачали со страницы соревнования.

In [0]:
DATA_PATH = 'gdrive/My Drive/Colab Notebooks (1)/'

---

In [0]:
!ls 'gdrive/My Drive/Colab Notebooks (1)/'

baseline.ipynb	catalogue.json	test_users.json
bookmarks.csv	ratings.csv	transactions.csv


`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах.

In [0]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}
catalogue = pd.DataFrame.from_dict(catalogue,'index').reset_index()
# av_hot - нужно для валидации. Эта колонка равна 1, если фильм или сериал может быть потребленным
catalogue['av_hot'] = catalogue['availability'].apply(lambda x: 1 if x == [] or 'subscription' in x else 0)
catalogue = catalogue.rename(index=str, columns={'index':'element_uid'})

In [0]:
catalogue.head()

,element_uid,type,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,attributes,av_hot
0,0,movie,[],80,2.912198e+07,0.575260,0,1.128332,0.654707,"[31115, 6713, 10906, 31116, 31117, 270, 24431,...",1
1,1,movie,"[purchase, rent]",120,6.610431e+06,0.773224,3,1.112014,0.654707,"[2786, 385, 2799, 3730, 886, 7, 11700, 42, 20,...",0
2,2,movie,[],80,1.315874e+07,0.699502,0,1.110127,0.680410,"[31442, 31443, 31444, 31445, 113, 31446, 42, 3...",1
3,3,series,[],20,4.157712e+07,0.702981,0,1.141929,0.654707,"[34361, 34362, 23033, 14887, 270, 20089, 43, 25]",1
4,4,movie,"[purchase, rent, subscription]",70,3.999579e+07,0.626596,8,1.130076,0.592716,"[26732, 26733, 26734, 9367, 7792, 336, 26735, ...",1


 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

---

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [0]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

---

`transactions.csv` — список всех транзакций за определённый период времени

In [0]:
%%time
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

CPU times: user 7.74 s, sys: 755 ms, total: 8.5 s
Wall time: 15.4 s


In [0]:
transactions.head(3)

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
0,3336,5177,S,4.430518e+07,4282,0,50
1,481,593316,S,4.430518e+07,2989,0,11
2,4128,262355,S,4.430518e+07,833,0,50


In [0]:
np.unique(transactions['consumption_mode'].values)

array(['P', 'R', 'S'], dtype=object)

 - `element_uid` — идентификатор элемента
 - `user_uid` — идентификатор пользователя
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке)
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке
 - `watched_time` — число просмотренных по транзакции секунд
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр

In [0]:
# Здесь создаем тестовое множество пользователей. Пока это будет хранится в validate_user

# Найдем пользователей, которые потребили услугу. Это как раз то, что надо предсказать
# Первое условие - пользователь купил или арендовал фильм
temp = transactions[(transactions['consumption_mode']=='R') | (transactions['consumption_mode']=='P')]

# Сохраним все пары пользователь-фильм в переменную
validate_user_tran = temp[['user_uid','element_uid']]

# Найдем теперь пары, где фильм был просмотрен более чем наполовину, если это было доступно бесплатно или по подписке
films = catalogue[(catalogue['type']=='movie') & (catalogue['av_hot']==1)][['element_uid','duration']]
temp = pd.merge(transactions, films, left_on='element_uid',right_on='element_uid', how='inner')
temp_films = temp[temp['watched_time']>0.5*temp['duration']]
temp_films = temp_films[['user_uid','element_uid']]

# аналогично для сериалов
series = catalogue[(catalogue['type']=='series') & (catalogue['av_hot']==1)][['element_uid','duration']]
temp_s = pd.merge(transactions, series, left_on='element_uid',right_on='element_uid', how='inner')
temp_s = temp_s[temp_s['watched_time']>0.33*temp_s['duration']]
temp_s = temp_s[['user_uid','element_uid']]

temp = pd.concat([temp_films,temp_s,validate_user_tran])

test = temp

In [0]:
test_part = test[:10000]


In [0]:

temp.head()

,user_uid,element_uid
0,5177,3336
1,349202,3336
2,117582,3336
3,523833,3336
4,571064,3336


---

`ratings.csv` содержит информацию о поставленных пользователями оценках

In [0]:
%%time
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

CPU times: user 258 ms, sys: 13.5 ms, total: 271 ms
Wall time: 534 ms


In [0]:
ratings.head(3)

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07


 - `rating` — поставленный пользователем рейтинг (от `0` до `10`)

---

`bookmarks.csv` содержит информацию об элементах, добавленных пользователями в список «Избранное»

In [0]:
%%time
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

CPU times: user 454 ms, sys: 22.9 ms, total: 477 ms
Wall time: 1.07 s


In [0]:
bookmarks.head(3)

,user_uid,element_uid,ts
0,301135,7185,4.430516e+07
1,301135,4083,4.430516e+07
2,301135,10158,4.430516e+07


In [0]:
bookmarks['rating'] = 1
_bookmarks = bookmarks[['rating', 'user_uid', 'element_uid']]
_bookmarks.head()

,rating,user_uid,element_uid
0,1,301135,7185
1,1,301135,4083
2,1,301135,10158
3,1,301135,2693
4,1,301135,2181


### Решение

Для начала построим список элементов, которые тестовые пользователи уже купили или посмотрели по подписке: они не смогут купить их второй раз, а просмотр по подписке второй раз маловероятен, поэтому мы захотим отфильтровать такие элементы из финального ответа.

Точно так же можно поступить и с рейтингами и добавлениями в избранное, если это будет казаться правильным.

In [0]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

100%|██████████| 9643012/9643012 [00:17<00:00, 543250.94it/s]


---

Для примера мы воспользуемся методом K ближайших соседей, реализованным в библиотеке `implicit`. В качестве данных используем только информацию о рейтингах.

Необходимо построить разреженную матрицу, где строкам будут соответствовать элементы, столбцам — пользователи, а на пересечении пользователя и элемента будет находиться количественная оценка степени их взаимодействия, если таковое имело место.

Не забудем добавить `1` к рейтингу, чтобы избежать деления на ноль во время вычисления `tf-idf`.

In [0]:
ratings['user_uid'] = ratings['user_uid'].astype('category')
ratings['element_uid'] = ratings['element_uid'].astype('category')

ratings_matrix = sp.coo_matrix(
    (ratings['rating'].astype(np.float32) + 1,
        (
            ratings['element_uid'].cat.codes.copy(),
            ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()


In [0]:
ratings.head()


,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07
3,359870,1364,10,4.430506e+07
4,359870,3578,9,4.430506e+07


In [0]:
_ratings = ratings[ratings['rating']>4]

In [0]:
_ratings['rating'] = np.where(_ratings['rating']>=1, 1, 0)
_ratings.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,user_uid,element_uid,rating,ts
0,571252,1364,1,4.430517e+07
1,63140,3037,1,4.430514e+07
2,443817,4363,1,4.430514e+07
3,359870,1364,1,4.430506e+07
4,359870,3578,1,4.430506e+07


In [0]:
_transaction = transactions[['element_uid', 'user_uid','watched_time']]
_transaction = _transaction[_transaction['watched_time']>600]
_transaction['rating'] = 1

In [0]:
np.unique(_ratings['rating'].values)

array([1])

In [0]:
_ratings.shape[0]

438790

In [0]:
all_ratings = pd.concat([_ratings[['user_uid', 'element_uid', 'rating']], _bookmarks[['user_uid', 'element_uid', 'rating']]])
all_ratings = pd.concat([all_ratings, _transaction])

all_ratings['user_uid'] = all_ratings['user_uid'].astype('category')
all_ratings['element_uid'] = all_ratings['element_uid'].astype('category')

ratings_matrix = sp.coo_matrix(
    (all_ratings['rating'].astype(np.float32) + 1,
        (
            all_ratings['element_uid'].cat.codes.copy(),
            all_ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()

In [0]:
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)

Sparsity: 0.001770


Обучить модель крайне просто.

In [0]:
!pip install implicit==0.3.7

In [0]:
%%time
from implicit.nearest_neighbours import TFIDFRecommender
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=64)

# model = TFIDFRecommender()
model.fit(ratings_matrix)

RuntimeError: ignored

In [0]:
!im

---

In [0]:
ratings_matrix_T = ratings_matrix.T.tocsr()


Отображения из оригинальной категории во внутреннюю пригодится нам в дальнейшем.

In [0]:
user_uid_to_cat = dict(zip(
    all_ratings['user_uid'].cat.categories,
    range(len(all_ratings['user_uid'].cat.categories))
))

In [0]:
element_uid_to_cat = dict(zip(
    all_ratings['element_uid'].cat.categories,
    range(len(all_ratings['element_uid'].cat.categories))
))

In [0]:
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

---

В метод `model.recommend` мы передаём идентификатор пользователя, который получаем обратным преобразованием из категории, транспонированную матрицу взаимодействий, число необходимых рекомендаций и список элементов, которые мы договорились фильтровать из ответа.

Возвращает метод список пар (`element_cat`, `score`), отсортированный по вторым элементам. Из него необходимо достать все первые элементы пар и из категории преобразовать их к `element_uid`.

**Важно:** Не все тестовые пользователи есть в `ratings.csv` и не все из них есть в `transactions.csv`. Используя только один источник данных мы не можем построить полное предсказание. Такой ответ с неполным числом пользователей бдет принят системой, но при вычислении средней метрики метрика для отсутствующих пользователей будет принята равной нулю.

In [0]:
result = {}

for user_uid in tqdm.tqdm(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(all_ratings['element_uid'].cat.categories[i]) for i, _ in recs]

In [0]:
def eval(model, test):
  ev = []
  counter = 0
  for user in np.unique(test['user_uid'].values):
    counter +=1
    y_true = test[test['user_uid']==5177].values[:,1] 


    user_cat = user_uid_to_cat[user_uid]
    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
#score()

    y_pred = np.array(recs)[:,0]
    summ = 0
    for i in range(1,21):
      summ += len(np.intersect1d(y_pred[:i],y_true))/i
    ev.append(summ)
    if counter == 10000:
      break
    if counter % 1000 == 0:
      print(counter)
  return np.mean(np.array(ev))

eval(model,test)

In [0]:
len(result)

49920

Используя только информацию о рейтингах мы смогли построить предсказание для `13251` из `50000` тестовых пользователей. Ровно в таком виде ответы и стоит сохранить для отправки.

In [0]:
from os.path import join
with open(join(DATA_PATH, 'answer.json'), 'w') as f:
    json.dump(result, f)

NameError: ignored